In [1]:
from scipy.spatial.transform import Rotation as R
import json
import numpy as np

def create_homogeneous_matrix(xyz, quaternion):
    rotation_matrix = R.from_quat(quaternion).as_matrix()
    T = np.eye(4) 
    T[:3, :3] = rotation_matrix
    T[:3, 3] = xyz 

    return T

def pose_inv(pose):
    R = pose[:3, :3]
    T = np.eye(4)
    T[:3, :3] = R.T
    T[:3, 3] = - R.T @ np.ascontiguousarray(pose[:3, 3])
    return T


K = np.load("handeye/intrinsics_d415.npy")
T_WC = np.load("handeye/T_WC_head.npy")
DIR = "scissor"

with open(f"experiments/{DIR}/demo_bottlenecks.json") as f:
    dbn = json.load(f)
    
T_GRIP_EEF = create_homogeneous_matrix([0, 0, 0.136], [0, 0, 0, 1])
demo_waypoints = np.vstack([dbn[key] for key in dbn.keys()])
demo_waypoints

array([[ 0.50418126,  0.09185436,  0.48143709, -0.99837866,  0.01344973,
        -0.05489271,  0.00677801],
       [ 0.49349562, -0.20686429,  0.41968363, -0.9836049 , -0.08057539,
         0.04566163,  0.15473856],
       [ 0.45431572,  0.05573993,  0.28884844,  0.99737963,  0.05747579,
         0.03869391,  0.02081296],
       [ 0.48795565, -0.09312451,  0.28185395, -0.99430351,  0.0169949 ,
         0.01044858,  0.10470206],
       [ 0.45431572,  0.14573993,  0.28884844,  0.99737963,  0.05747579,
         0.03869391,  0.02081296]])

In [8]:
import copy

def project3D(new_pose, ori_pose):
    project_pose = copy.deepcopy(new_pose)
    project_pose[2] = ori_pose[2]

    ori_euler = R.from_quat(ori_pose[3:]).as_euler("xyz")
    new_euler = R.from_quat(new_pose[3:]).as_euler("xyz")

    new_euler[0] = ori_euler[0]
    new_euler[1] = ori_euler[1]

    project_pose[3:] = R.from_euler("xyz", new_euler).as_quat()

    return project_pose

project_pose = project3D([0.5122576949427623, 0.2079057347792264, 0.47840045944896903, 0.9452851874707641, 0.3255858373266077, -0.011332219402534116, -0.017359656796922612], demo_waypoints[0])

AttributeError: 'numpy.ndarray' object has no attribute 'as_quat'

In [6]:
from trajectory_utils import pose_inv

def apply_transformation_to_waypoints(waypoints_np, delta_R, reverse=False, project3D=False):
    """
    Apply a transformation to a list of end-effector poses using NumPy vectorization.
    """

    # Separate translations and rotations
    translations = waypoints_np[:, :3]
    rotations = waypoints_np[:, 3:]

    # Convert quaternions to rotation matrices
    waypoint_rot_matrices = R.from_quat(rotations).as_matrix()

    # Construct 4x4 matrices for waypoints
    RW_matrices = np.zeros((waypoints_np.shape[0], 4, 4))
    RW_matrices[:, :3, :3] = waypoint_rot_matrices
    RW_matrices[:, :3, 3] = translations
    RW_matrices[:, 3, 3] = 1

    # Apply the transformation
    transformed_matrices =  RW_matrices @ delta_R if reverse else delta_R @ RW_matrices

    delta_eef =  @ transformed_matrices

    # Extract translations and rotations from transformed matrices
    transformed_translations = transformed_matrices[:, :3, 3]
    transformed_rotations = R.from_matrix(transformed_matrices[:, :3, :3]).as_quat()


    return transformed_matrices

T_delta_world = np.array([[ 0.79857538, -0.60179267, -0.011088  ,  0.28067213],
                        [ 0.60165335,  0.79864132, -0.01361215, -0.38310653],
                        [ 0.01704703,  0.00419919,  0.99984587, -0.00643008],
                        [ 0.        ,  0.        ,  0.        ,  1.        ]])

res = apply_transformation_to_waypoints(demo_waypoints, T_delta_world)
res

[[[ 0.80166556  0.59743358 -0.02013547  0.12090894]
  [-0.59703313  0.79854408 -0.07667335  0.10163424]
  [-0.02972817  0.07348792  0.99685293  0.00928547]
  [ 0.          0.          0.          1.        ]]

 [[ 0.79912659  0.57874845 -0.16262508  0.28719692]
  [-0.57245     0.81519079  0.08811913  0.09033893]
  [ 0.18356928  0.02267638  0.98274519 -0.05047731]
  [ 0.          0.          0.          1.        ]]

 [[ 0.80004568  0.59940666  0.02526969  0.13654073]
  [-0.59693278  0.7995416  -0.06636633  0.14128604]
  [-0.05998459  0.03801178  0.99747529  0.01526113]
  [ 0.          0.          0.          1.        ]]

 [[ 0.79854691  0.59181523 -0.10989801  0.23743796]
  [-0.59115714  0.80546126  0.04201652  0.06416333]
  [ 0.1133846   0.03141482  0.9930544  -0.02103465]
  [ 0.          0.          0.          1.        ]]

 [[ 0.80004568  0.59940666  0.02526969  0.08082076]
  [-0.59693278  0.7995416  -0.06636633  0.15316788]
  [-0.05998459  0.03801178  0.99747529  0.01124641]
  [ 

array([[[ 0.80887007,  0.58083392,  0.09143945,  0.62268341],
        [ 0.57427787, -0.81378487,  0.08921382, -0.01295889],
        [ 0.12623046, -0.01965073, -0.99180629,  0.48391332],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.68170208,  0.68391294, -0.25989491,  0.79460154],
        [ 0.7300928 , -0.65891683,  0.18108871, -0.2571164 ],
        [-0.04740022, -0.31319594, -0.94850489,  0.42073284],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.72011363,  0.68705635,  0.09690164,  0.60673085],
        [ 0.68770943, -0.72528781,  0.0318329 , -0.06918159],
        [ 0.09215258,  0.04371687, -0.99478477,  0.29035264],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.81723114,  0.56181937, -0.12842217,  0.72325795],
        [ 0.5762621 , -0.79949035,  0.16952041, -0.16773609],
        [-0.00743243, -0.21254219, -0.97712362,  0.28330757],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],



ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 7)

In [6]:
res = apply_transformation_to_waypoints(demo_waypoints, T_delta_world, project3D=True)
res[1]

[0.794601542455787,
 -0.25711640018581905,
 0.4196836324630228,
 0.9072269032405044,
 0.3884717989473198,
 -0.09239512018035448,
 -0.1322578890884605]

In [62]:
point_2Ds = []

for i in range(len(demo_waypoints)):
    point3d = pose_inv(T_WC) @ T_GRIP_EEF @ create_homogeneous_matrix(demo_waypoints[i, :3], demo_waypoints[i, 3:])
    
    # Project the 3D point onto the image plane
    point_image_homogeneous = np.dot(K, point3d[:3, 3])
    
    # Normalize the coordinates to get the 2D image point
    point_2D = point_image_homogeneous[:2] / point_image_homogeneous[2]

    point_2Ds.append(point_2D)

point_2Ds

[array([1134.18602041,   63.35327591]),
 array([591.82082126, 156.4616502 ]),
 array([967.88171055, 318.41246776]),
 array([762.80761621, 349.90925078]),
 array([1093.95793212,  319.87692355])]

In [63]:
from PIL import Image, ImageDraw

head_rgb = Image.open(f"data/{DIR}/demo_head_rgb.png")

# Create a drawing context
draw = ImageDraw.Draw(head_rgb)

for point_2D in point_2Ds:
    x, y, r = point_2D[0], point_2D[1], 5
    # Draw the circle (ellipse) with bounding box [(x-r, y-r), (x+r, y+r)]
    draw.ellipse((x-r, y-r, x+r, y+r), outline="red", width=3)


# Save or show the image
head_rgb.show()  # To display the image